In [1]:
using Distributed
addprocs(4);

@everywhere using CSV, DataFrames, GLM, Distributions, LinearAlgebra, Statistics
@everywhere using BSON, Dates
@everywhere using XLSX

@everywhere using RCall
@everywhere R"library(tvReg)"
@everywhere R"library(forecast)"

@everywhere include("tvPersistence_functions_Restat_Rev5.jl");

@everywhere using CSV, DataFrames, GLM, Distributions, LinearAlgebra, Statistics, Random
@everywhere using BSON, Dates, Plots, StatsBase

┌ Warning: RCall.jl: Loading required package: Matrix
│ Funded by the Horizon 2020. Framework Programme of the European Union.
│ 
└ @ RCall /Users/lukasvacha/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Loading required package: Matrix
│ Funded by the Horizon 2020. Framework Programme of the European Union.
│ 
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Loading required package: Matrix
│ Funded by the Horizon 2020. Framework Programme of the European Union.
│ 
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Loading required package: Matrix
│ Funded by the Horizon 2020. Framework Programme of the European Union.
│ 
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Loading required package: Matrix
│ Funded by the Horizon 2020. Framework Programme of the European Union.
│ 
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Registered S3 method overwritten by 'quantm

In [2]:

#load all volatility  
@everywhere data = CSV.read("RV.csv", DataFrame)
@everywhere data = Matrix(data[:,2:end])
@everywhere data = 100.0*data;

# Median volatility
@everywhere data=median(data,dims=2);

@everywhere X = data
@everywhere T= length(X)
@everywhere AR=22
@everywhere T -= AR

# compute AR22 coeff
@everywhere gamma0 = hcat(ones(T), hcat([X[AR-(k-1):end-k] for k in 1:AR]...)) \ X[AR+1:end]

# get errors
@everywhere eHatR = X[AR+1:end] .- (hcat(ones(T), hcat([X[k:end-AR+k-1, :] for k in 1:AR]...)) * gamma0);
@everywhere uHatR = eHatR ./ std(eHatR);

########## HORIZON 22 ##########################

@everywhere horizon=22

### Chose parameters
@everywhere maxAR					= 15;   # original with AR=5
# @everywhere kernel_width_for_const 	= 0.05; # deterministic (constant) forecast
@everywhere kernel_width_for_const 	= 0.1; # deterministic (constant) forecast
@everywhere kernel_width_IRF		= 0.2;  # IRFs on errors
@everywhere kernel_width_forecast 	= 0.4;  # forecast of constant
@everywhere AR_lag_forecast 		= 1;	# AR lag choice for forecast of constant
@everywhere kernel_width_HAR		= 0.4; 	# kernel for TVP-AR3 TVP-HAR model 
@everywhere JMAX=7;  

# @everywhere maxAREWD				= 10;   # original with AR=5
# @everywhere JMAXEWD=6;   

@everywhere tt=1000;

### Bootstrap h=1

@everywhere B=30;

@everywhere burnIn = 0
@everywhere errors=[]
@everywhere forecasts=[]

println("Horizon 22 Bootstrap started. ", now())
flush(stdout)

for i=1:B
    
	println("B ", i)
	flush(stdout) 

@everywhere Xb = vcat(vcat([X[rand(1:T), :] for k=1:AR]...), zeros(T-AR+1 + burnIn));    

# i.i.d. bootstrap index draw ### !!! I took the simple variant !!!
@everywhere ind = rand(1:T, T + burnIn + 1)
@everywhere ubR = uHatR[ind];
@everywhere ebR = ubR*std(eHatR);

    for t in AR:T + burnIn
       Xb[t] = [1;[Xb[t-k+1,:] for k in 1:AR]...]'* gamma0 + ebR[t + 1];
    end

	@everywhere rvfx = Xb;

	@everywhere fcast_length = length(rvfx)-tt-21-horizon; #length of forecast sample

	(horizon_forecast_HAR,Error_HAR) = HAR(rvfx,tt,fcast_length,horizon);

	# out0=hcat(pmap(i -> EWD_parallel(i,rvfx,tt,maxAR,JMAX,horizon), 0:(fcast_length-1))...);
	# horizon_forecast_EWD=out0[1,:];
	# RV_h=out0[2,:];
	# Error_EWD=out0[3,:];

	out=hcat(pmap(i -> AR3_tvLS_parallel(i,rvfx,tt,fcast_length,horizon,kernel_width_HAR), 0:(fcast_length-1))...);
	horizon_forecast_AR3tv=out[1,:];
	Error_AR3tv=out[2,:];

	out=hcat(pmap(i -> HAR_tvLS_parralel(i,rvfx,tt,fcast_length,horizon,kernel_width_HAR), 0:(fcast_length-1))...);
	horizon_forecast_HARtv=out[1,:];
	Error_HARtv=out[2,:];

	out=hcat(pmap(i -> EWD_tvLS_parallel(i,rvfx,tt,maxAR,JMAX,horizon,kernel_width_for_const,kernel_width_IRF,kernel_width_forecast,AR_lag_forecast), 0:(fcast_length-1))...);
	horizon_forecast_EWDtv=out[1,:];
	Error_EWDtv=out[2,:];

    res=[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv]
    # res=[Error_AR3tv Error_HARtv]
	push!(errors,res)
end

output_bson = Dict("errors" => errors, "tt" => tt,"horizon" => horizon)
BSON.bson("30_all_vol_boot_$tt-$horizon.bson", output_bson)

println("Horizon 22 Bootstrap completed. ", now())
flush(stdout)

Horizon 22 Bootstrap started. 2024-11-25T23:16:42.293
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
B 27
B 28
B 29
B 30
Horizon 22 Bootstrap completed. 2024-11-26T03:37:43.423


In [4]:
# Kernel smoothing function
function kernel_smooth(data::Vector{Float64}, bandwidth::Float64)
    n = length(data)
    smoothed = zeros(Float64, n)

    # Gaussian kernel function
    kernel(x) = exp(-0.5 * x^2) / sqrt(2 * π)

    for i in 1:n
        weights = [kernel((i - j) / bandwidth) for j in 1:n]
        weights /= sum(weights)  # Normalize weights
        smoothed[i] = sum(weights .* data)
    end

    return smoothed
end

function SED_smooth(s1,s2,band)
    data=s1.^2 .- s2.^2;
    R"time <- 1:length($data)
    tv_model <- tvLM($data ~ time,bw=$band)
    smoothed_values <- fitted(tv_model)"
    return convert(Array{Float64},R"smoothed_values");
end

SED_smooth (generic function with 1 method)

In [3]:
# [Error_HAR Error_AR3tv Error_HARtv Error_EWDtv]
### NEW with RW bechmark ####

tt=1000     

test1 = BSON.load("results_RV_SP500_final_predictions_$tt-1.bson")
test5 = BSON.load("results_RV_SP500_final_predictions_$tt-5.bson")
test22 = BSON.load("results_RV_SP500_final_predictions_$tt-22.bson")

res1=test1["forecasts"];
res5=test5["forecasts"];
res22=test22["forecasts"];

names=["HAR" "EWD" "TV-AR3" "TV-HAR" "TV-EWD"]

RW1 = BSON.load("results_RV_SP500_RW_predictions_$tt-1.bson")
RW5 = BSON.load("results_RV_SP500_RW_predictions_$tt-5.bson")
RW22 = BSON.load("results_RV_SP500_RW_predictions_$tt-22.bson")
RW1=RW1["forecasts"];
RW5=RW5["forecasts"];
RW22=RW22["forecasts"];

res1all = [hcat(res1[i][:, 1], RW1[i][], res1[i][:, 2:end]) for i in 1:496];
res5all = [hcat(res5[i][:, 1], RW5[i][], res5[i][:, 2:end]) for i in 1:496];
res22all = [hcat(res22[i][:, 1], RW22[i][], res22[i][:, 2:end]) for i in 1:496];
lev1=0.05
lev5=0.05
lev22=0.05

yl=0.0001;
yh=0.0004;

datum = CSV.read("dates.csv", DataFrame)
datum=Matrix(datum);

tt=1000 
datpos=datum[tt+22+1:end]
sel=[[1; size(res1[1])[1]] [1;751] [1751;2001]]

k=1;   ### Change of period here ###
t1=sel[1,k]
t2=sel[2,k] 

##### ERRORS FOR ALL (RW as a bench ) #####
#err_all1=[hcat([Float64.(trim(res1all[j][t1:t2,1].-res1all[j][t1:t2,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6]
#err_all5=[hcat([Float64.(trim(res5all[j][t1:t2-5,1].-res5all[j][t1:t2-5,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6]
#err_all22=[hcat([Float64.(trim(res22all[j][t1:t2-22,1].-res22all[j][t1:t2-22,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6]

rv_true1=hcat([Float64.(res1all[j][t1:t2,1]) for j in 1:496]...)
rv_true5=hcat([Float64.(res5all[j][t1:t2-5,1]) for j in 1:496]...)
rv_true22=hcat([Float64.(res22all[j][t1:t2-22,1]) for j in 1:496]...);

err_all1=[hcat([Float64.(winsor(res1all[j][t1:t2,1].-res1all[j][t1:t2,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6]
err_all5=[hcat([Float64.(winsor(res5all[j][t1:t2-5,1].-res5all[j][t1:t2-5,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6]
err_all22=[hcat([Float64.(winsor(res22all[j][t1:t2-22,1].-res22all[j][t1:t2-22,1+jj],prop=lev1)) for j in 1:496]...) for jj=1:6];
err_all1[6][isnan.(err_all1[6])].=0.0
err_all5[6][isnan.(err_all5[6])].=0.0
err_all22[6][isnan.(err_all22[6])].=0.0;

# names=["RW" "HAR" "EWD" "TV-AR3" "TV-HAR" "TV-EWD"];

# h1 server B=100, AR22, All four models

In [287]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test1b = BSON.load("0_1_vol_boot_$tt-1.bson")
res1b=test1b["errors"];
B=100
SED=[SED_smooth(res1b[i][:,1],res1b[i][:,2],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

0.0003467599642181169

In [277]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test1b = BSON.load("0_1_vol_boot_$tt-1.bson")
res1b=test1b["errors"];
B=100
SED=[SED_smooth(res1b[i][:,1],res1b[i][:,3],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

4.011045083750926e-6

In [278]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test1b = BSON.load("0_1_vol_boot_$tt-1.bson")
res1b=test1b["errors"];
B=100
SED=[SED_smooth(res1b[i][:,1],res1b[i][:,4],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

1.2019772866130706e-5

# h5 server, B=30, AR22, All four models

In [283]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test5b = BSON.load("vol_boot_$tt-5.bson")
res5b=test5b["errors"];
B=30
SED=[SED_smooth(res5b[i][:,1],res5b[i][:,2],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

3.1122818805228614e-5

In [279]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test5b = BSON.load("vol_boot_$tt-5.bson")
res5b=test5b["errors"];
B=30
SED=[SED_smooth(res5b[i][:,1],res5b[i][:,3],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

1.0934609909422118e-5

In [280]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test5b = BSON.load("vol_boot_$tt-5.bson")
res5b=test5b["errors"];
B=30
SED=[SED_smooth(res5b[i][:,1],res5b[i][:,4],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])


2.402437424495225e-6

# 30 h22 komp All, AR22

In [13]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test22b = BSON.load("30_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,2],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

4.8143324467630284e-5

In [14]:
# Load Bootstrap

bw=0.05
tt=1000     
test22b = BSON.load("30_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,3],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

3.164509128040154e-5

In [15]:
# Load Bootstrap

bw=0.05
tt=1000     
test22b = BSON.load("30_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,4],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

2.481788564607729e-5

# 30 h22 komp All, AR1

In [16]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test22b = BSON.load("30_AR1_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,2],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

6.82026230827422e-6

In [17]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test22b = BSON.load("30_AR1_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,3],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

5.688913526381037e-6

In [18]:
# Load Bootstrap
###[Error_HAR Error_AR3tv Error_HARtv Error_EWDtv] ###
bw=0.05
tt=1000     
test22b = BSON.load("30_AR1_all_vol_boot_$tt-22.bson")
res22b=test22b["errors"];

B=30
SED=[SED_smooth(res22b[i][:,1],res22b[i][:,4],bw) for i=1:B];
median([quantile!([SED[j][jj] for j=1:B], 0.95) for jj=100:2130])

4.251738505230786e-6